In [1]:
import pandas as pd
import numpy as np
import math
# read and import excel to dataframe
dfFactory = pd.read_excel (r'NTTDevExam-v2.1.Mid-1.xlsx', sheet_name='FactoryPlan')
dfStore = pd.read_excel (r'NTTDevExam-v2.1.Mid-1.xlsx', sheet_name='StorePlan')
all_ActualQtyFactory = dfFactory.groupby('Factory')['ActualQty'].sum().sum()
print (dfFactory)
print (dfStore)

   Factory  PlanQty  ActualQty
0     5001      110         54
1     5005      101        200
2     5012      172        150
3     5042      110         94
4     5046       79         92
5     5113      116         84
     Factory  Store  PlanQty  ActualQty
0       5001   1127        2        NaN
1       5001   1154        2        NaN
2       5001   1164        2        NaN
3       5001   1220        2        NaN
4       5001   1224        2        NaN
..       ...    ...      ...        ...
143     5113   6577        6        NaN
144     5113   6592        7        NaN
145     5113   6599        7        NaN
146     5113   6611        8        NaN
147     5113   6638        7        NaN

[148 rows x 4 columns]


In [2]:
# keep Foctory ID in array which has ActualQty than PlanQty
ActualMorePlan = []
for index, row in dfFactory.iterrows():
    if(row['ActualQty']>=row['PlanQty']):
        ActualMorePlan.append(row['Factory'])
ActualMorePlan

[5005, 5046]

In [3]:
# array of Factory Id which is not enough ActualQty for PlanQty
ActualLessPlan = []
ActualLessPlan = dfFactory["Factory"].tolist()
# not intersact (use not intersact ActualLessPlan)
ActualLessPlan = set(ActualLessPlan) ^ set(ActualMorePlan)
ActualLessPlan

{5001, 5012, 5042, 5113}

In [4]:
# loop for setting the ActualQty in store dataframe, which only the row having enough PlanQty following ActualMorePlan array
for foctoryId in ActualMorePlan:
#     set ActualQty in store
    dfStore.loc[dfStore['Factory'] == foctoryId, 'ActualQty'] = dfStore['PlanQty']
#     update actualQty 
    dfFactory.loc[dfFactory['Factory'] == foctoryId, 'ActualQty'] = dfFactory['ActualQty']-dfFactory['PlanQty']
print (dfFactory)
print (dfStore.groupby('Factory')['ActualQty'])

   Factory  PlanQty  ActualQty
0     5001      110         54
1     5005      101         99
2     5012      172        150
3     5042      110         94
4     5046       79         13
5     5113      116         84


In [5]:
# set the demand weight for each Factory
dfFactory['Weight']=dfFactory['ActualQty']/dfFactory['PlanQty']
dfFactory

,Factory,PlanQty,ActualQty,Weight
0,5001,110,54,0.490909
1,5005,101,99,0.980198
2,5012,172,150,0.872093
3,5042,110,94,0.854545
4,5046,79,13,0.164557
5,5113,116,84,0.724138


In [6]:
# sort demand weight from high to low (The high weight, the high priority)
dfFactory = dfFactory.sort_values(by=['Weight'], ascending=False)
dfFactory.index = range(len(dfFactory))
dfFactory

,Factory,PlanQty,ActualQty,Weight
0,5005,101,99,0.980198
1,5012,172,150,0.872093
2,5042,110,94,0.854545
3,5113,116,84,0.724138
4,5001,110,54,0.490909
5,5046,79,13,0.164557


In [7]:
# function for set ActualQty to link with PlanQty by adding from others Factory, and update the leftovers 
# of each Factory
def updateActualQtyForCeil(dfStore,dfFactory,ActualMorePlan,store):
    add_next = -1
#     loop for setting ActualQty of Factory which is enough for PlanQty
    for foctoryMoreId in ActualMorePlan:
#         check leftover of Factory ActualQty does not destitute 
        if dfFactory.loc[dfFactory['Factory'] == foctoryMoreId,'ActualQty'].iat[0] == 0:
                continue
#         check is it have the preavious value
        if (add_next > 0):
            wantMore = add_next
        else:
            wantMore = leftoversQty
#         get leftover of Factory ActualQty
        sumActualQty_More_Factory = dfFactory.loc[dfFactory['Factory'] == foctoryMoreId,'ActualQty'].iat[0]
#         check leftover of Factory ActualQty is it enough for the requirement 
        if(sumActualQty_More_Factory<wantMore):
#             the reqiurment is not enough only this ActualQty of Factory
            add_next = wantMore - sumActualQty_More_Factory
            wantMore = sumActualQty_More_Factory
            dfFactory.loc[dfFactory['Factory'] == foctoryMoreId, 'ActualQty'] = 0
        else:
            add_next = -1
            dfFactory.loc[dfFactory['Factory'] == foctoryMoreId, 'ActualQty'] = sumActualQty_More_Factory - wantMore
#         add new special requirment to the dataframe
        new_row = {'Factory':foctoryMoreId,'Store':store,'PlanQty': 0,'ActualQty':wantMore}
        dfStore = dfStore.append(new_row,ignore_index=True)
        break
    return dfStore, dfFactory

In [8]:
# loop for set the ActualQty of Factory which is not enough for PlanQty.
# By using the demand weight and ceiling value. From the ceiling value, it makes the sum of ActualQty
# more than the real ActualQty 
for index,storeRow in dfStore.iterrows():
#     Check the factory is in the array of ActualQty less than PlanQty
    if(storeRow['Factory'] in ActualLessPlan):
#         intit the current factory id
        foctoryId = storeRow['Factory']
        weight = dfFactory.loc[dfFactory['Factory'] == foctoryId,'Weight'].iat[0]
#        calculate ActualQty with weight and cailling 
        realActualQty = math.ceil(storeRow['PlanQty'] * weight)
#        get the sum of the ActualQty of current foctory in current time
        sumActualQty = dfStore.groupby('Factory')['ActualQty'].sum().get(key =foctoryId)
#        get the leftover of Factory ActualQty
        sumActualQty_Foctory = dfFactory.loc[dfFactory['Factory'] == foctoryId, 'ActualQty'].iat[0]
#        check the leftover is enough for ActualQty
        if( (realActualQty+sumActualQty) <= sumActualQty_Foctory ):
            dfStore.at[index,'ActualQty'] = realActualQty
        else:
            leftoversQty = (sumActualQty+realActualQty)-sumActualQty_Foctory
            dfStore.at[index,'ActualQty'] = sumActualQty_Foctory-sumActualQty
#             call function to update the store dataframe
            dfResult = updateActualQtyForCeil(dfStore,dfFactory,ActualMorePlan,storeRow['Store'])
            dfStore = dfResult[0]
            dfFactory = dfResult[1]
    else:
        continue

In [9]:
dfStore.groupby('Factory')['ActualQty'].sum()

Factory
5001.0     54.0
5005.0    138.0
5012.0    150.0
5042.0     94.0
5046.0     79.0
5113.0     84.0
Name: ActualQty, dtype: float64

In [10]:
# set the percent of complete order between PlanQty and ActualQty
dfStore['CompleteOrder%']=(dfStore['ActualQty']*100)/dfStore['PlanQty']
dfStore

,Factory,Store,PlanQty,ActualQty,CompleteOrder%
0,5001.0,1127.0,2.0,1.0,50.0
1,5001.0,1154.0,2.0,1.0,50.0
2,5001.0,1164.0,2.0,1.0,50.0
3,5001.0,1220.0,2.0,1.0,50.0
4,5001.0,1224.0,2.0,1.0,50.0
...,...,...,...,...,...
152,5005.0,6457.0,0.0,5.0,inf
153,5005.0,6498.0,0.0,3.0,inf
154,5005.0,6551.0,0.0,6.0,inf
155,5005.0,6611.0,0.0,3.0,inf


In [11]:
dfFactory

,Factory,PlanQty,ActualQty,Weight
0,5005,101,62,0.980198
1,5012,172,150,0.872093
2,5042,110,94,0.854545
3,5113,116,84,0.724138
4,5001,110,54,0.490909
5,5046,79,13,0.164557


In [12]:
# update the ActualQty of Factory which is not enough for PlanQty from leftovers. 
# This function updates the ActualQty of Factory which is not enough for PlanQty and
# ordering from complete order and demand wieght.
def updateActualQtyForleftovers(dfStore,dfFactory,ActualMorePlan,ActualLessPlan):
    print ("Case 2")
#     sort dataframe
    dfStore = dfStore.sort_values(by=['CompleteOrder%'], ascending=True)
    dfFactory = dfFactory.sort_values(by=['Weight'], ascending=False)
    dfFactory.index = range(len(dfFactory))
#     loop the store dataframe row 
    for index,storeRow in dfStore.iterrows():
        if(storeRow['Factory'] in ActualLessPlan):
            wantMore = storeRow['PlanQty'] - storeRow['ActualQty']
            storeId = storeRow['Store']
#             skip the uneeded ActualQty
            if(wantMore == 0): continue
            add_next = -1;
#             set and update the ActualQty of Factory which is enough for PlanQty
            for foctoryIdMore in ActualMorePlan:
                leftovers = dfFactory.loc[dfFactory['Factory'] == foctoryIdMore, 'ActualQty'].iat[0]
#             Check is there are leftovers
                if(leftovers == 0 and foctoryIdMore !=ActualMorePlan[-1]):
                    continue
                elif(leftovers == 0 and foctoryIdMore == ActualMorePlan[-1]):
                    return dfStore
                else:
#                     check there is not enough ActualQty from the preavious Factory
                    if(add_next>0):
                        wantMore = add_next
#                     check the leftover is engound for the current and update the leftovers
                    if (leftovers >= wantMore):
                        dfFactory.loc[dfFactory['Factory'] == foctoryIdMore, 'ActualQty'] = leftovers - wantMore
                        add_next = -1
                    elif (leftovers < wantMore):
                        dfFactory.loc[dfFactory['Factory'] == foctoryIdMore, 'ActualQty'] = 0
                        wantMore = leftovers
                        add_next = wantMore - leftovers
#                     update or set the ActualQt
                    oldSet_ActualQty = dfStore.loc[(dfStore['Factory'] == foctoryIdMore) & (dfStore['Store'] == storeId), 'ActualQty']
                    if(len(oldSet_ActualQty)== 0): 
                        new_row = {'Factory':foctoryIdMore,'Store':storeId,'PlanQty': 0,'ActualQty':wantMore}
                        dfStore = dfStore.append(new_row, ignore_index=True)
                    else :
                        wantMore = wantMore + oldSet_ActualQty
                        dfStore.loc[(dfStore['Factory'] == foctoryIdMore)& (dfStore['Store'] == storeId), 'ActualQty'] = wantMore
                    break
        else:
            continue

In [13]:
# remove function which to update the store dataframe, after ceilling makes the ActualQty is more than the real one.
def remove(dfStore,dfFactory,ActualMorePlan,ActualLessPlan):
    print ("Case 1")
#     sort the complete ordering 
    dfFactory = dfFactory.sort_values(by=['Weight'], ascending=True)
    dfFactory.index = range(len(dfFactory))
    leftovers_ActualQty = dfStore.groupby('Factory')['ActualQty'].sum().sum()-all_ActualQtyFactory
#     loop for remove the data that is over 
    for index,storeRow in dfStore.iterrows():
        if(storeRow['Factory'] in ActualMorePlan):
            leftovers_ActualQty = leftovers_ActualQty - storeRow['ActualQty']
            dfFactory.drop([index], inplace = True)
        if(leftovers_ActualQty<=0):
            break
    return dfStore

In [14]:
print (dfFactory.groupby('Factory')['ActualQty'].sum())
print (dfStore.groupby('Factory')['ActualQty'].sum())

Factory
5001     54
5005     62
5012    150
5042     94
5046     13
5113     84
Name: ActualQty, dtype: int64
Factory
5001.0     54.0
5005.0    138.0
5012.0    150.0
5042.0     94.0
5046.0     79.0
5113.0     84.0
Name: ActualQty, dtype: float64


In [15]:
print (dfFactory.groupby('Factory')['ActualQty'].sum())
print (dfStore.groupby('Factory')['ActualQty'].sum())

Factory
5001     54
5005     62
5012    150
5042     94
5046     13
5113     84
Name: ActualQty, dtype: int64
Factory
5001.0     54.0
5005.0    138.0
5012.0    150.0
5042.0     94.0
5046.0     79.0
5113.0     84.0
Name: ActualQty, dtype: float64


In [16]:
leftovers_ActualQty=all_ActualQtyFactory-dfStore.groupby('Factory')['ActualQty'].sum().sum()
leftovers_ActualQty

75.0

In [17]:
# last updating the ActualQty
# 1. the sum of ActualQty less than the real one
# 2. the sum of ActualQty more than the real one
# 3. complete ActualQty
# 4. Error
if leftovers_ActualQty<0:
    dfStore = remove(dfStore,dfFactory,ActualMorePlan,ActualLessPlan)
elif leftovers_ActualQty>0:
    dfStore = updateActualQtyForleftovers(dfStore,dfFactory,ActualMorePlan,ActualLessPlan)
elif leftovers_ActualQty>0:
    print ("Complete - manage store (Left over)")
else:
    print ("Error Sort data")

Case 2


In [18]:
dfStore.groupby('Factory')['ActualQty'].sum()

Factory
5001.0     54.0
5005.0    200.0
5012.0    150.0
5042.0     94.0
5046.0     92.0
5113.0     84.0
Name: ActualQty, dtype: float64

In [19]:
dfStore = dfStore.drop('CompleteOrder%', axis=1)

In [20]:
# !pip install openpyxl

In [21]:
# export result of the store dataframe
# dfStore.to_excel(r'Path where the exported excel file will be stored\result_store.xlsx',index = False)
dfStore.to_excel(r'result_NTTDevExam-v2.1.Mid.xlsx',index = False)